In [1]:
import matplotlib
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
import os
import random
from scipy import ndimage 
from sklearn.cluster import KMeans
import imageio

################  Image_Partition  ##################################


def Image_Partition(img_list):
    vectors=[]
    patch_x = 64
    patch_y = 64
    
    for filtered_img in img_list:
        #count=0
        for x in range(0,filtered_img.shape[0]+1 - patch_x, patch_x):
            #count2=0
            for y in range(0,filtered_img.shape[1]+1 - patch_y, patch_y):
                patch = filtered_img[x:x+patch_x,y:y+patch_y]
                '''
                 vertical_projection
                '''
                vector = np.sum(patch, axis = 1)
                vectors.append(vector)
            #    count2+=1
           # count+=1
            #partition_part.append(window)


         #   print(count2)
          ##  print(count) 
       # print(filtered_img.shape[1]+1)
    return vectors

########################## paper  #################################
#print(filtered_img.shape[0])

########################  Schmid filters  ############################
def Make_filter(kernal_size,tau,sigma):   
    hsup=(kernal_size-1)/2
    m1 = np.linspace(-hsup, hsup,kernal_size) 
    m2 = np.linspace(-hsup, hsup,kernal_size)
    [x,y]=np.meshgrid(m1,m2)
    mul=((x*x)+(y*y))
    r= np.sqrt(mul)
    f=(np.cos(2*r*(np.pi*tau/sigma))*(np.exp(-(r*r)/(2*sigma*sigma))))
    f -= np.mean(f)     
    f=f/np.sum(np.abs(f))
    #print(f)
    return f


def Schmid_Filter(training_img):

    kernal_size=49        
    img_list=[]
    for i in range(5):    
        if i==0:   
            fimg =cv2.filter2D(training_img, cv2.CV_8UC3,Make_filter(kernal_size,2,1)) 
        elif i==1:  
            fimg =cv2.filter2D(training_img, cv2.CV_8UC3,Make_filter(kernal_size,4,2)) 
        elif i==2:
            fimg =cv2.filter2D(training_img, cv2.CV_8UC3,Make_filter(kernal_size,6,2)) 
        elif i==3:
            fimg =cv2.filter2D(training_img, cv2.CV_8UC3,Make_filter(kernal_size,8,2)) 
        elif i==4:
            fimg =cv2.filter2D(training_img, cv2.CV_8UC3,Make_filter(kernal_size,10,2)) 
           #you can determine the constatnt  
           #fimg =ndimage.filters.convolve(img, Make_filter(kernal_size,10,2), mode='constant',cval=0.0) 
           #mode='same' mean the output keep the same as innput
           #fimg =ndimage.filters.convolve(img, Make_filter(kernal_size,10,2), mode='same') 
        img_list.append(fimg)
    return img_list  
        






########################  Gabor Filtes  ############################


#kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, phi, ktype=cv2.CV_32F)
def Gabor_Filter(training_img):
    img_list=[]
    for i in range(4):
        if i==0:
            kernel = cv2.getGaborKernel((15, 15), 2, 0, 4, .3, 0, ktype=cv2.CV_32F)     
            fimg = cv2.filter2D(training_img, cv2.CV_8UC3, kernel)
        elif i==1:  
            kernel = cv2.getGaborKernel((15, 15), 1, 0, 4, .4, 0, ktype=cv2.CV_32F)     
            fimg = cv2.filter2D(training_img, cv2.CV_8UC3, kernel) 
        elif i==2:
            kernel = cv2.getGaborKernel((15, 15), 2, np.pi/2, 8, .3, 0, ktype=cv2.CV_32F)     
            fimg = cv2.filter2D(training_img, cv2.CV_8UC3, kernel)
        elif i==3:
            kernel = cv2.getGaborKernel((15, 15), 1, np.pi/2, 8, .4, 0, ktype=cv2.CV_32F)     
            fimg = cv2.filter2D(training_img, cv2.CV_8UC3, kernel)

        
        img_list.append(fimg)
    return img_list  


#######################  read dataset and get random two image from each class  ########################## 

dataset_path= 'EXACT09_CT'

list_of_classes_name = os.listdir(dataset_path)

#print(dirs)

''' 
#get 2 imgae randome from each class  
'''
list_training_vectors=[]

count_random_images = 2    
for class_name in list_of_classes_name:
    list_all_images_in_class=os.listdir(dataset_path+'\\'+class_name)    
    for i in range(count_random_images):
        image_name = random.choice(list_all_images_in_class)
        image_path = os.path.join(dataset_path,class_name,image_name)
        image = cv2.imread(image_path)
#        assert not isinstance(im,type(None)), 'image not found'
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        gabor_output=Gabor_Filter(image)
        schmid_output=Schmid_Filter(image)
        vectors=Image_Partition(gabor_output+schmid_output)
        #print(image.shape)
        list_training_vectors.append(vectors)



all_training_vectors=np.array(list_training_vectors) 

all_training_vectors=all_training_vectors.reshape((all_training_vectors.shape[0]*all_training_vectors.shape[1]),all_training_vectors.shape[2])


'''
#third step
#learn the dictionary
'''
#k-means++  = selects initial cluster centers for k-mean clustering in a smart way to speed up
#n_init = Number of time the k-means algorithm will be run with different centroid seeds. 
#The final results will be the best output of n_init consecutive runs in terms of inertia.

# random_state Determines random number generation for centroid initialization

### The default values are n_init=10 and max_iter=300. 
## This means the initial centroids will be chosen 10 times, 
## and each run will use up to 300 iterations. 
## The best out of those 10 runs will be the final result.
n_dic=200

kmeans = KMeans(n_clusters=n_dic, init='k-means++' ,max_iter=300, n_init=10, random_state=None)
try:
    kmeans.fit(all_training_vectors)
except ValueError as e:
    print(e)




'''
convert all images in dataset into patchs then into vectors
'''

list_images_path=[]
list_vectors=[]
num_of_img=0



for class_name in list_of_classes_name:
    list_all_images_in_class=os.listdir(dataset_path+'\\'+class_name) 
    for i in list_all_images_in_class:
        num_of_img +=1
        image_path = os.path.join(dataset_path,class_name,i)
        list_images_path.append(image_path)
        image = cv2.imread(image_path)
       # assert not isinstance(im,type(None)), 'image not found'
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
        gabor_output=Gabor_Filter(image)
        schmid_output=Schmid_Filter(image)  
        vectors=Image_Partition(gabor_output+schmid_output)

        list_vectors.append(vectors)


    
all_dataset_vectors=np.array(list_vectors) 
   
all_dataset_vectors = all_dataset_vectors.reshape((all_dataset_vectors.shape[0]*all_dataset_vectors.shape[1]),all_dataset_vectors.shape[2])



'''
  #four step
  #obtain the histogram from bag of features(BoVW)
  #compute features for each images
'''

list_histogram_of_each_image_in_dataset =[]
classification=np.arange(200)
for i in range(num_of_img):
    #predicting n_patches of an image
    #classification array of prediction class for each patch
    classification=kmeans.predict(all_dataset_vectors[(i*576): (i*576)+576])
    
    #computes histogarme and append in the array
    ##densitybool, optional If False, the result will contain the number of samples in each bin. 
    #If True, the result is the value of the probability density function at the bin
    image_histogram,_=np.histogram(classification, bins=range(n_dic+1),density=True)
    #np.histogram return array of image_histogram and bins_edges of (length(hist)+1) 
    list_histogram_of_each_image_in_dataset.append(image_histogram)
    
list_histogram_of_each_image_in_dataset = np.array(list_histogram_of_each_image_in_dataset)
print('Number of images and features =',list_histogram_of_each_image_in_dataset.shape)  

'''
calculate the ARP
Query part
'''

final_ARP=[]
ARP_of_class=[]

def hist_match(query_hist,image_hist):
    result=0
    for i in range(query_hist.size):
        if query_hist[i]<=image_hist[i]:
            result=result+query_hist[i]
        else:
            result=result+image_hist[i] 
    return result

Number of images and features = (675, 200)


In [2]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askopenfile
from PIL import Image, ImageTk

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib
matplotlib.use("TkAgg")

In [8]:
#GUI of Prediction   
window = tk.Tk()
menu = Menu(window)
window.config(menu=menu)
filemenu = Menu(menu)
menu.add_cascade(label='File', menu=filemenu)
filemenu.add_command(label='New')
filemenu.add_command(label='Open...')
filemenu.add_separator()
filemenu.add_command(label='Exit', command=menu.quit)
helpmenu = Menu(menu)
menu.add_cascade(label='Help', menu=helpmenu)
helpmenu.add_command(label='About')

ourMessage ='Welcome to our GUI where we classify the Lungs Images'
messageVar = Message(window, width=300,fg="white",text = ourMessage)
messageVar.config(bg='gray')
messageVar.pack( )

our ='------------------------------------------------------------ '
messageVa = Message(window, width=300,fg="white",text = our)
messageVa.config(bg='gray')
messageVa.pack( )


our ='Ahmed Samir Afifi '
messageVa = Message(window, width=300,fg="white",text = our)
messageVa.config(bg='gray')
messageVa.pack( )

our ='Asmaa Shawqy Youins'
messageVa = Message(window, width=300,fg="white",text = our)
messageVa.config(bg='gray')
messageVa.pack( )

our ='Aya sherief Elasmar'
messageVa = Message(window, width=300,fg="white",text = our)
messageVa.config(bg='gray')
messageVa.pack( )

our ='Hekmat Gmal AbuElnnasr'
messageVa = Message(window, width=300,fg="white",text = our)
messageVa.config(bg='gray')
messageVa.pack( )

our ='Youmna Bahaa Eldean'
messageVa = Message(window, width=300,fg="white",text = our)
messageVa.config(bg='gray')
messageVa.pack( )

our ='Nada Reda Afifi'
messageVa = Message(window, width=300,fg="white",text = our)
messageVa.config(bg='gray')
messageVa.pack( )


# Set window title
window.configure(background='gray')
window.title('This Form to Classify the Images of Lungs Tumor')
window.geometry('900x300')

window.resizable(False, False)

txtbox= tk.Entry(window,width=100)
txtbox= tk.Entry(window,width=50)
txtbox.pack()

btn = tk.Button(window,text="Classify",width=20,command = lambda:upload_file())
btn.pack()



def upload_file():
    f_types = [('Jpg Files', '*.jpg'),
               ('PNG Files', '*.png')]   # type of files to select
    filename = tk.filedialog.askopenfilename(multiple=True, filetypes=f_types)
    txtbox.insert(0,filename)
    class_name = str(list(filename))[-16:-13].replace("/","")
    print(class_name)
    list_all_images_in_class = os.listdir(dataset_path+'/'+class_name)

    image_path = str(list(filename))[2:-2]
    print(image_path)
    average_precision=0
    list_all_images_in_class=os.listdir(dataset_path+'/'+class_name)
    query_image = cv2.imread(image_path)
   # assert not isinstance(im,type(None)), 'image not found'
    query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2GRAY)  
    gabor_output=Gabor_Filter(query_image)
    schmid_output=Schmid_Filter(query_image)
    #get the features vectors of the query 
    query_vectors=Image_Partition(gabor_output+schmid_output)
    #convert list to array
    query_vectors = np.array(query_vectors)
    query_classify=kmeans.predict(query_vectors)
    #get the histograam of the query
    query_histogram,_=np.histogram(query_classify,bins=range(n_dic+1),density=True)

   #claculte the amount similarity between query histogam and all images histogram
    #and append in list_amount_similarity the result of the similarity for each image and the path of this image 
    list_amount_similarity = []
    for x in range(num_of_img):    
        similarity_result=hist_match(list_histogram_of_each_image_in_dataset[x],query_histogram)
        list_amount_similarity.append((similarity_result,str(list_images_path[x])))




    total_image_retrieved=10
    #ordered the amount of similarity in the list fom small to larg and take the 
    #number of top image you want to retreive
    top_image_retrieved = sorted(list_amount_similarity,reverse=True)[:total_image_retrieved]



    #imgs =[]
    path_retrieved_image=[]
    retrieved_image_class=[]
    similarity_result=[]
    count =0
    #get path of each reterievd image and append to path_retrieved_image
    #and get claass of this reteived image and append to retrieved_image_class
    for similarity_amount, image_path in top_image_retrieved:
        similarity_result.append(similarity_amount) 
        path_retrieved_image.append(image_path)
        class_path=os.path.dirname(image_path)
        retrieved_image_class.append(os.path.basename(class_path)) 








    plt.figure(figsize=(12,8))
    plt.subplot(441);plt.imshow(query_image)
    plt.title('Query');plt.axis('off')

    retrieved_image =[]
    relevant_images=0 

    #calculate the pecision for each query 
    #and then add the pecision to the average_precision
    if int(class_name)==1: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.2f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==1:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision      

    elif int(class_name) ==2:
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==2:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision    


    elif int(class_name) ==3: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])

            if int(retrieved_image_class[i])==3:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision     

    elif int(class_name) ==4: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==4:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision         

    elif int(class_name) ==5: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==5:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision     

    elif int(class_name) ==6: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==6:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision        

    elif int(class_name) ==7: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==7:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision         

    elif int(class_name) ==8:
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==8:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision         

    elif int(class_name) ==9: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==9:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision         

    elif int(class_name) ==10:
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==10:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision        

    elif int(class_name) ==11: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==11:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision       

    elif int(class_name) ==12: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==12:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision        

    elif int(class_name) ==13: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==13:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision        

    elif int(class_name) ==14: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==14:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision      

    elif int(class_name) ==15: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==15:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision       

    elif int(class_name) ==16:
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==16:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision        


    elif int(class_name) ==17: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==17:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision        

    elif int(class_name)==18: 
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==18:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision       

    elif int(class_name) ==19:
        for i in range(total_image_retrieved):
            retrieved_image.append(imageio.imread(path_retrieved_image[i]))
            plt.subplot(4,4,i+2); plt.imshow(retrieved_image[i])
            plt.title('%d, %.4f,%d' % (i,similarity_result[i],int(retrieved_image_class[i])));plt.axis('off')  

            if int(retrieved_image_class[i])==19:
                relevant_images +=1
        precision=relevant_images/total_image_retrieved
        average_precision+=precision    


    number_of_images_in_class=len(list_all_images_in_class)
    #use the average_precision for each class to calculate ARP for each class and append to ARP_of_class
    ARP_of_class.append((100/number_of_images_in_class)*average_precision)

    

window.mainloop()  # Keep the window open

2
C:/Users/farag/DSS/EXACT09_CT/2/3133230.jpg
9
C:/Users/farag/DSS/EXACT09_CT/9/3133454.jpg
